<a href="https://colab.research.google.com/github/Gecrz/DGMD-S-14-Final-Project/blob/main/Unsucessful%20Models/Transfer%20learning%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#download the data using the below code into your local setup

import requests
from tqdm import tqdm

url = 'http://www.inf.ufpr.br/vri/databases/BreaKHis_v1.tar.gz'
file_path = 'BreaKHis_v1.tar.gz'

response = requests.get(url, stream=True)

if response.status_code == 200:
    total_size = int(response.headers.get('content-length', 0))
    progress_bar = tqdm(total=total_size, unit='B', unit_scale=True)

    with open(file_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                file.write(chunk)
                progress_bar.update(len(chunk))

    progress_bar.close()
    print('File downloaded successfully.')
else:
    print('Failed to download the file.')


100%|██████████| 4.27G/4.27G [04:12<00:00, 16.9MB/s]

File downloaded successfully.


In [ ]:
# get all the 400x png files into a folder
import tarfile
import os
import shutil
import io

# Define the tar file
tar_file_path = 'BreaKHis_v1.tar.gz'

# Define the directory to store the extracted files
output_dir = 'extracted_images_400x'
os.makedirs(output_dir, exist_ok=True)

# Open the tar file
tar = tarfile.open(tar_file_path)

# Iterate over each member in the tar file
for member in tar.getmembers():
    if '400X' in member.name and member.name.endswith('.png'):
        # Use extractfile() to get a file-like object for the file data
        file_data = tar.extractfile(member)

        if file_data is not None:
            # Get the base filename (without directories)
            base_filename = os.path.basename(member.name)
            # Define the target file path in the output_dir
            target_path = os.path.join(output_dir, base_filename)

            # Write the file data to the new file
            with open(target_path, 'wb') as f:
                shutil.copyfileobj(file_data, f)

# Close the tar file
tar.close()


In [ ]:
!ls extracted_images_400x | head -n 20


SOB_B_A-14-22549AB-400-001.png
SOB_B_A-14-22549AB-400-002.png
SOB_B_A-14-22549AB-400-003.png
SOB_B_A-14-22549AB-400-004.png
SOB_B_A-14-22549AB-400-005.png
SOB_B_A-14-22549AB-400-006.png
SOB_B_A-14-22549AB-400-007.png
SOB_B_A-14-22549AB-400-008.png
SOB_B_A-14-22549AB-400-009.png
SOB_B_A-14-22549AB-400-010.png
SOB_B_A-14-22549AB-400-011.png
SOB_B_A-14-22549AB-400-012.png
SOB_B_A-14-22549AB-400-013.png
SOB_B_A-14-22549AB-400-014.png
SOB_B_A-14-22549AB-400-015.png
SOB_B_A-14-22549AB-400-016.png
SOB_B_A-14-22549AB-400-017.png
SOB_B_A-14-22549AB-400-018.png
SOB_B_A-14-22549AB-400-019.png
SOB_B_A-14-22549AB-400-020.png


In [ ]:
#creating a dataframe that has the patient id, file path, type

import os
import pandas as pd

# Define the directory where the files are stored
directory = 'extracted_images_400x'

# Get the list of file names
filenames = os.listdir(directory)

# Create a DataFrame from the file names
df = pd.DataFrame(filenames, columns=['filename'])

# Extract the type (M or B), patient ID and image ID from the filename
df['type'] = df['filename'].str.split('_', expand=True)[1]
df['patient_ID'] = df['filename'].str.split('-', expand=True)[2]

# Display the DataFrame
df


,filename,type,patient_ID
0,SOB_M_DC-14-2773-400-033.png,M,2773
1,SOB_B_PT-14-29315EF-400-012.png,B,29315EF
2,SOB_M_DC-14-15792-400-008.png,M,15792
3,SOB_M_DC-14-20636-400-008.png,M,20636
4,SOB_M_DC-14-11951-400-002.png,M,11951
...,...,...,...
1815,SOB_M_DC-14-5694-400-011.png,M,5694
1816,SOB_M_DC-14-16188-400-021.png,M,16188
1817,SOB_M_PC-14-12465-400-013.png,M,12465
1818,SOB_B_F-14-25197-400-039.png,B,25197


In [ ]:
import numpy as np

# Get a list of unique patient IDs for each type
malignant_ids = df[df['type'] == 'M']['patient_ID'].unique()
benign_ids = df[df['type'] == 'B']['patient_ID'].unique()

# Function to split patient IDs into train, validation, and test
def split_ids(ids):
    np.random.shuffle(ids)
    train_size = int(len(ids) * 0.8)
    val_size = int(len(ids) * 0.1)
    train_ids = ids[:train_size]
    val_ids = ids[train_size:train_size + val_size]
    test_ids = ids[train_size + val_size:]
    return train_ids, val_ids, test_ids

# Split patient IDs for each type
train_m_ids, val_m_ids, test_m_ids = split_ids(malignant_ids)
train_b_ids, val_b_ids, test_b_ids = split_ids(benign_ids)

# Concatenate train, validation, and test patient IDs
train_patient_ids = np.concatenate([train_m_ids, train_b_ids])
val_patient_ids = np.concatenate([val_m_ids, val_b_ids])
test_patient_ids = np.concatenate([test_m_ids, test_b_ids])

# Get corresponding dataframes
train_df = df[df['patient_ID'].isin(train_patient_ids)]
val_df = df[df['patient_ID'].isin(val_patient_ids)]
test_df = df[df['patient_ID'].isin(test_patient_ids)]


In [ ]:
print('file name is', 'train_df')
for column in train_df.columns:
    print(column, train_df[column].nunique())


print('file name is', 'val_df')
for column in val_df.columns:
    print(column, val_df[column].nunique())


print('file name is', 'test_df')
for column in test_df.columns:
    print(column, test_df[column].nunique())


file name is train_df
filename 1384
type 2
patient_ID 64
file name is val_df
filename 211
type 2
patient_ID 7
file name is test_df
filename 225
type 2
patient_ID 10


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# Define image size and path
img_size = (700, 460)
batch_size = 32
directory = 'extracted_images_400x'

# Prepare data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # assuming 80-20 split for train-val

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=directory,
    x_col='filename',
    y_col='type',
    target_size=img_size,
    class_mode='binary',
    batch_size=batch_size,
    subset='training')

val_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=directory,
    x_col='filename',
    y_col='type',
    target_size=img_size,
    class_mode='binary',
    batch_size=batch_size,
    subset='validation')

# Load pre-trained VGG16 model
base_model = VGG16(input_shape=img_size + (3,), include_top=False, weights='imagenet')

# Freeze the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a new model on top
model = Sequential([
    base_model,
    Flatten(),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, validation_data=val_generator, epochs=7)


Found 1108 validated image filenames belonging to 2 classes.
Found 276 validated image filenames belonging to 2 classes.
Epoch 1/7
35/35 [==============================] - 43s 1s/step - loss: 1.0448 - accuracy: 0.6471 - val_loss: 0.6426 - val_accuracy: 0.7029
Epoch 2/7
35/35 [==============================] - 41s 1s/step - loss: 0.3075 - accuracy: 0.8673 - val_loss: 0.3940 - val_accuracy: 0.8333
Epoch 3/7
35/35 [==============================] - 41s 1s/step - loss: 0.1966 - accuracy: 0.9540 - val_loss: 0.4002 - val_accuracy: 0.7899
Epoch 4/7
35/35 [==============================] - 42s 1s/step - loss: 0.1584 - accuracy: 0.9486 - val_loss: 0.3264 - val_accuracy: 0.8514
Epoch 5/7
35/35 [==============================] - 41s 1s/step - loss: 0.0836 - accuracy: 0.9838 - val_loss: 0.3263 - val_accuracy: 0.8587
Epoch 6/7
35/35 [==============================] - 40s 1s/step - loss: 0.0693 - accuracy: 0.9928 - val_loss: 0.3176 - val_accuracy: 0.8514
Epoch 7/7
35/35 [============================

In [ ]:
# Prepare data generator for test set
test_datagen = ImageDataGenerator(rescale=1./255)  # rescale the pixel values to [0,1]

test_generator = test_datagen.flow_from_dataframe(
    test_df,  # your test dataframe
    directory=directory,
    x_col='filename',
    y_col='type',
    target_size=img_size,
    class_mode='binary',
    batch_size=batch_size,
    shuffle=False)  # prevent shuffle for precise evaluation

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Found 225 validated image filenames belonging to 2 classes.
8/8 [==============================] - 9s 998ms/step - loss: 0.7159 - accuracy: 0.7333
Test Loss: 0.7159224152565002
Test Accuracy: 0.7333333492279053


In [ ]:
from IPython.display import Javascript

# Play a beep sound
Javascript('new Audio("https://www.soundjay.com/button/beep-07.wav").play()')
